Zubin Sheth's CCS on Sports Analytics
First attempt to quanitfy cricket players's 'form' using RFM as a metric.

Enter inputs below.

In [18]:
TEAM = 'Royal Challengers Bangalore'
SEASON = 2017
PLAYER = 'V Kohli'
#PLAYER = 'N Rana'
favorable_score = 30

In [19]:
# Print all outputs in a block - not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# standard imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [20]:
# Source match level data 
matches_data=pd.read_csv('../data/matches.csv')
matches_data.head(5)
#matches_data.dtypes

# Getting all 2017 matches from the dataset for RCB home and away matches
ids2017 = matches_data.loc[(matches_data['season'] == SEASON) & ((matches_data['team1'] == TEAM) | (matches_data['team2'] == TEAM))]
ids2017 = ids2017['id'].unique()
ids2017

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


array([ 1,  5,  8, 12, 17, 20, 27, 30, 33, 37, 42, 45, 55])

In [21]:
# Source ball by ball level data
ball_data=pd.read_csv('../data/deliveries.csv')
#list(ball_data)
#ball_data.iloc[0]
ball_data.head(1)
#ball_data['batsman'].unique()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN


In [22]:
# Getting all player (Virat Kohli) data
vk = ball_data.loc[ ( ball_data['match_id'].isin(ids2017) )]
vk = vk.loc[ ( vk['batting_team'] == TEAM)]
vk = vk.loc[ ( vk['batsman'] == PLAYER )]
vk = vk[['match_id', 'inning', 'over', 'batsman_runs']]
vk.head(5)

,match_id,inning,over,batsman_runs
2590,12,1,1,0
2591,12,1,1,1
2593,12,1,1,0
2594,12,1,1,1
2597,12,1,2,0


In [23]:
# Sum up all runs per match
vk = pd.pivot_table(vk, index=['match_id'], values=['batsman_runs'], aggfunc=np.sum)
vk['match_num'] = np.arange(len(vk))
vk['match_num'] = len(vk) - vk['match_num']
vk

,batsman_runs,match_num
match_id,,
12,62,10
17,28,9
20,64,8
27,0,7
30,10,6
33,55,5
37,20,4
42,6,3
45,5,2


In [24]:
# Func to check for eligibility based on threshold
# Return 1 if greater than condition matches else 0
def score(row, col, threshold):
    if(row[col] >= threshold):
        return 1
    else:
        return 0

In [25]:
# For all eligible scores (eligible is score > favorable score)
vk['eligible'] = vk.apply(lambda row: score(row, 'batsman_runs', favorable_score), axis=1)
vk['recency_score'] = vk['eligible'] / vk['match_num']
vk

,batsman_runs,match_num,eligible,recency_score
match_id,,,,
12,62,10,1,0.100
17,28,9,0,0.000
20,64,8,1,0.125
27,0,7,0,0.000
30,10,6,0,0.000
33,55,5,1,0.200
37,20,4,0,0.000
42,6,3,0,0.000
45,5,2,0,0.000


In [26]:
# recency score
final_recency_score = np.sum(vk['recency_score'] / len(vk['recency_score']))
final_recency_score

0.14250000000000002

In [27]:
# frequency score
final_frequency_score = np.sum(vk['eligible'] / len(vk['eligible']))
final_frequency_score

0.4

In [28]:
# monetory score
final_monetory_score = np.sum(vk['eligible'] * vk['batsman_runs']) / len(vk['eligible'])
final_monetory_score

23.9

In [29]:
# final rfm score
rfm_score = final_recency_score + final_frequency_score + final_monetory_score
rfm_score

24.4425